# 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert

from datetime import datetime, timedelta

In [ ]:
# 셀레니움으로 url 주소 크롬 열기
driver = webdriver.Chrome()
url = 'https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do'

driver.get(url)
time.sleep(1)

# 초기 날짜 지정
start_day = datetime.strptime("2023-01-01", '%Y-%m-%d')

while True:

    start_day_str = start_day.strftime('%Y-%m-%d')

    # 조회기간 시작일 비우기
    driver.find_element(By.NAME,'sSearchFrom').clear()
    time.sleep(0.5)
    Alert(driver).accept()
    time.sleep(0.5)
    
    # 조회기간 마지막일 비우기
    driver.find_element(By.NAME,'sSearchTo').clear()
    time.sleep(0.5)
    Alert(driver).accept()
    time.sleep(0.5)
    
    # 조회기간 시작일 값 입력
    driver.find_element(By.NAME,'sSearchFrom').send_keys(start_day_str)
    time.sleep(0.5)
    driver.find_element(By.XPATH,'/html/body').click()
    time.sleep(0.5)

    # 조회 버튼 클릭
    driver.find_element(By.CSS_SELECTOR,'div.wrap_btn .btn_blue').click()
    time.sleep(2)

    # 엑셀 파일 다운로드
    driver.find_element(By.CSS_SELECTOR,'div.tar a:nth-child(1)').click()
    time.sleep(2)

    # 자바스크립트 확인
    Alert(driver).accept()
    time.sleep(3)
    
    # 시작일 변경
    start_day = start_day + timedelta(days=7)
    
    # 최종일 지정
    if start_day == datetime.strptime("2023-07-23", '%Y-%m-%d'):
        break

# 하나의 파일로 만들기

In [ ]:
import pandas as pd
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [ ]:
col_list = ['순위', '영화명', '개봉일', '매출액', '매출액점유율', 
            '매출액증감(전일대비)', '매출액증감율(전일대비) ','누적매출액 ', 
            '관객수 ', '관객수증감(전일대비) ', '관객수증감율(전일대비) ', 
            '누적관객수 ','스크린수 ', '상영횟수 ', '대표국적 ', '국적 ', 
            '제작사 ', '배급사 ', '등급 ', '장르 ','감독 ', '배우 ','기준일']

In [ ]:
df1 = pd.DataFrame(columns = col_list)

for file in glob(r'c:/2nd_project/Data/movie_data/2023년(1~7)/*.xlsx'):
    base = pd.read_excel(file, engine='openpyxl')
    
    try: 

        ii = base[base["Unnamed: 4"].isnull()].index[range(7,len(base[base["Unnamed: 4"].isnull()].index),3)]
        ii_lis = [7]

        for i in range(len(ii)):
            ii_lis.append(int(ii[i])+3)

        df2 = pd.DataFrame(columns = col_list)

        for jj in range(len(ii_lis)):
            if jj != len(ii_lis)-1:
                df = base.iloc[ii_lis[jj]:ii_lis[jj+1]-4]
                df["기준일"] = datetime.strptime(base.iloc[ii_lis[jj]-3][0].split("●")[1].split("(")[0],'%Y년 %m월 %d일')
                df.columns = col_list
                df2 = pd.concat([df,df2],axis=0)

            else:
                df = base.iloc[ii_lis[jj]:]
                df["기준일"] = datetime.strptime(base.iloc[ii_lis[jj]-3][0].split("●")[1].split("(")[0],'%Y년 %m월 %d일')
                df.columns = col_list
                df2 = pd.concat([df,df2],axis=0)
        df1 = pd.concat([df1,df2],axis=0)
    
    except:
        pass

In [ ]:
datetime.strptime("2023-07-31", '%Y-%m-%d') - datetime.strptime("2023-01-01", '%Y-%m-%d')

# 파일 확인

In [ ]:
df1

In [ ]:
df1.to_csv("c:/2nd_project/Data/movie_data/2023_0107.csv", index=False)

In [21]:
import pandas as pd
df1 = pd.read_csv('c:/2nd_project/Data/movie_data/2023_0107.csv')
df1

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
0,1,아바타: 물의 길,2022-12-14 00:00:00,2970060027,0.401,-438191291.0,-0.129,109635532854,222534,-29192.0,...,4495,미국,미국,NaN,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,"액션,어드벤처,SF,스릴러",제임스 카메론,"조 샐다나,샘 워싱턴,시고니 위버,스티븐 랭,케이트 윈슬렛",2023-01-08
1,2,영웅,2022-12-21 00:00:00,1278854920,0.173,-86342045.0,-0.063,21864715710,125892,-7516.0,...,2746,한국,한국,"(주)제이케이필름,(주)씨제이이엔엠",(주)씨제이이엔엠,12세이상관람가,"드라마,뮤지컬",윤제균,"정성화,김고은,나문희,조재윤,배정남,이현우,박진주,권라희,지웅배",2023-01-08
2,3,더 퍼스트 슬램덩크,2023-01-04 00:00:00,1263213165,0.170,-107094091.0,-0.078,4358290178,120605,-8633.0,...,2660,일본,일본,NaN,(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,애니메이션,이노우에 다케히코,NaN,2023-01-08
3,4,장화신은 고양이: 끝내주는 모험,2023-01-04 00:00:00,843131365,0.114,-45301860.0,-0.051,2994435536,86726,-4442.0,...,2611,미국,미국,NaN,유니버설픽쳐스인터내셔널 코리아(유),전체관람가,애니메이션,조엘 크로포드,"안토니오 반데라스,셀마 헤이엑,올리비아 콜맨,하비 길렌,샘슨 카요,와그너 모라,안토...",2023-01-08
4,5,스위치,2023-01-04 00:00:00,520628516,0.070,-54198455.0,-0.094,2227685561,50992,-5590.0,...,2798,한국,한국,(주)하이브미디어코프,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,기타,마대윤,"권상우,오정세,이민정,박소이,김준,김미경,김명선,이서환,차시원,성병숙,김승훈,윤대열...",2023-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26614,77,요정,2022-12-01 00:00:00,6000,0.000,-6000.0,-0.500,29194997,1,-1.0,...,2,한국,한국,한국예술종합학교 미디어콘텐츠센터,"(주)싸이더스,(주)블루라벨픽쳐스",12세이상관람가,드라마,신택수,"류현경,김주헌,김신비,김금순,공상아,이종윤,오재균,이수형,맹주원,송덕호,강주리,강정민",2023-01-07
26615,77,질펀한 마사지:대담한 손스킬,2023-01-07 00:00:00,6000,0.000,6000.0,1.000,6000,1,1.0,...,1,일본,일본,NaN,(주)영진크리에이티브,청소년관람불가,"성인물(에로),드라마,멜로/로맨스",디렉터 O,NaN,2023-01-07
26616,87,더 게임,2022-12-08 00:00:00,0,0.000,0.0,1.000,4450699,0,0.0,...,1,헝가리,헝가리,NaN,알토미디어(주),15세이상관람가,"드라마,스릴러",페테르 퍼자카스,가브리엘라 하모리,2023-01-07
26617,87,뮤직 바이 시아,2022-12-15 00:00:00,0,0.000,-6000.0,-1.000,7801500,0,-1.0,...,2,미국,미국,NaN,(주)디오시네마,15세이상관람가,"드라마,뮤지컬",NaN,"케이트 허드슨,레슬리 오덤 주니어",2023-01-07


In [ ]:
imsi = df1.groupby("기준일")["영화명"].count().to_frame().reset_index()
imsi['기준연도'] = imsi['기준일'].apply(lambda x:x.year)
imsi['기준월'] = imsi['기준일'].apply(lambda x:x.month)
imsi['기준연도'].unique()

In [ ]:
 imsi['기준월'].unique()

In [22]:
df1[df1['순위'] == '합계']

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
89,합계,NaN,NaN,7410912792,NaN,NaN,NaN,303857312934,657781,NaN,...,17535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-08
171,합계,NaN,NaN,2341630062,NaN,NaN,NaN,275209161994,212816,NaN,...,15741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-09
263,합계,NaN,NaN,2222401628,NaN,NaN,NaN,277360285157,206439,NaN,...,15789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-10
365,합계,NaN,NaN,2183968263,NaN,NaN,NaN,283884614088,204310,NaN,...,15952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-11
467,합계,NaN,NaN,2028371611,NaN,NaN,NaN,314482104864,189482,NaN,...,15853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26245,합계,NaN,NaN,2639210645,NaN,NaN,NaN,314126234012,237122,NaN,...,14948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-03
26341,합계,NaN,NaN,3347381614,NaN,NaN,NaN,285772418041,313901,NaN,...,16651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-04
26434,합계,NaN,NaN,2972365524,NaN,NaN,NaN,276067650113,275666,NaN,...,16574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-05
26529,합계,NaN,NaN,3858996012,NaN,NaN,NaN,258096541386,341162,NaN,...,17106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-06


In [23]:
df_final = df1[df1['순위'] != '합계']
df_final

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
0,1,아바타: 물의 길,2022-12-14 00:00:00,2970060027,0.401,-438191291.0,-0.129,109635532854,222534,-29192.0,...,4495,미국,미국,NaN,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,"액션,어드벤처,SF,스릴러",제임스 카메론,"조 샐다나,샘 워싱턴,시고니 위버,스티븐 랭,케이트 윈슬렛",2023-01-08
1,2,영웅,2022-12-21 00:00:00,1278854920,0.173,-86342045.0,-0.063,21864715710,125892,-7516.0,...,2746,한국,한국,"(주)제이케이필름,(주)씨제이이엔엠",(주)씨제이이엔엠,12세이상관람가,"드라마,뮤지컬",윤제균,"정성화,김고은,나문희,조재윤,배정남,이현우,박진주,권라희,지웅배",2023-01-08
2,3,더 퍼스트 슬램덩크,2023-01-04 00:00:00,1263213165,0.170,-107094091.0,-0.078,4358290178,120605,-8633.0,...,2660,일본,일본,NaN,(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,애니메이션,이노우에 다케히코,NaN,2023-01-08
3,4,장화신은 고양이: 끝내주는 모험,2023-01-04 00:00:00,843131365,0.114,-45301860.0,-0.051,2994435536,86726,-4442.0,...,2611,미국,미국,NaN,유니버설픽쳐스인터내셔널 코리아(유),전체관람가,애니메이션,조엘 크로포드,"안토니오 반데라스,셀마 헤이엑,올리비아 콜맨,하비 길렌,샘슨 카요,와그너 모라,안토...",2023-01-08
4,5,스위치,2023-01-04 00:00:00,520628516,0.070,-54198455.0,-0.094,2227685561,50992,-5590.0,...,2798,한국,한국,(주)하이브미디어코프,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,기타,마대윤,"권상우,오정세,이민정,박소이,김준,김미경,김명선,이서환,차시원,성병숙,김승훈,윤대열...",2023-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26613,77,픽션들,2022-11-24 00:00:00,6000,0.000,-3000.0,-0.333,2952000,1,-2.0,...,1,한국,한국,경 필름,영화배급협동조합 씨네소파,15세이상관람가,드라마,장세경,"김권후,이태경,박종환,구자은,김민우,박인화,박병철,최원정,고명찬,이한주,주은실,황자...",2023-01-07
26614,77,요정,2022-12-01 00:00:00,6000,0.000,-6000.0,-0.500,29194997,1,-1.0,...,2,한국,한국,한국예술종합학교 미디어콘텐츠센터,"(주)싸이더스,(주)블루라벨픽쳐스",12세이상관람가,드라마,신택수,"류현경,김주헌,김신비,김금순,공상아,이종윤,오재균,이수형,맹주원,송덕호,강주리,강정민",2023-01-07
26615,77,질펀한 마사지:대담한 손스킬,2023-01-07 00:00:00,6000,0.000,6000.0,1.000,6000,1,1.0,...,1,일본,일본,NaN,(주)영진크리에이티브,청소년관람불가,"성인물(에로),드라마,멜로/로맨스",디렉터 O,NaN,2023-01-07
26616,87,더 게임,2022-12-08 00:00:00,0,0.000,0.0,1.000,4450699,0,0.0,...,1,헝가리,헝가리,NaN,알토미디어(주),15세이상관람가,"드라마,스릴러",페테르 퍼자카스,가브리엘라 하모리,2023-01-07


##### 영화명 / 개봉일 / 등급 / 장르 / 대표국적 / 국적 / 제작사 / 배급사 / 감독 / 배우

In [24]:
# df_final = df_final[['영화명', '개봉일', '등급 ', '장르 ', '대표국적 ', '국적 ',
#                     '제작사 ', '배급사 ','감독 ', '배우 ']]
df_final['개봉일'] = pd.to_datetime(df_final['개봉일']).dt.strftime('%Y-%m-%d')
df_final

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
0,1,아바타: 물의 길,2022-12-14,2970060027,0.401,-438191291.0,-0.129,109635532854,222534,-29192.0,...,4495,미국,미국,NaN,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,"액션,어드벤처,SF,스릴러",제임스 카메론,"조 샐다나,샘 워싱턴,시고니 위버,스티븐 랭,케이트 윈슬렛",2023-01-08
1,2,영웅,2022-12-21,1278854920,0.173,-86342045.0,-0.063,21864715710,125892,-7516.0,...,2746,한국,한국,"(주)제이케이필름,(주)씨제이이엔엠",(주)씨제이이엔엠,12세이상관람가,"드라마,뮤지컬",윤제균,"정성화,김고은,나문희,조재윤,배정남,이현우,박진주,권라희,지웅배",2023-01-08
2,3,더 퍼스트 슬램덩크,2023-01-04,1263213165,0.170,-107094091.0,-0.078,4358290178,120605,-8633.0,...,2660,일본,일본,NaN,(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,애니메이션,이노우에 다케히코,NaN,2023-01-08
3,4,장화신은 고양이: 끝내주는 모험,2023-01-04,843131365,0.114,-45301860.0,-0.051,2994435536,86726,-4442.0,...,2611,미국,미국,NaN,유니버설픽쳐스인터내셔널 코리아(유),전체관람가,애니메이션,조엘 크로포드,"안토니오 반데라스,셀마 헤이엑,올리비아 콜맨,하비 길렌,샘슨 카요,와그너 모라,안토...",2023-01-08
4,5,스위치,2023-01-04,520628516,0.070,-54198455.0,-0.094,2227685561,50992,-5590.0,...,2798,한국,한국,(주)하이브미디어코프,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,기타,마대윤,"권상우,오정세,이민정,박소이,김준,김미경,김명선,이서환,차시원,성병숙,김승훈,윤대열...",2023-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26613,77,픽션들,2022-11-24,6000,0.000,-3000.0,-0.333,2952000,1,-2.0,...,1,한국,한국,경 필름,영화배급협동조합 씨네소파,15세이상관람가,드라마,장세경,"김권후,이태경,박종환,구자은,김민우,박인화,박병철,최원정,고명찬,이한주,주은실,황자...",2023-01-07
26614,77,요정,2022-12-01,6000,0.000,-6000.0,-0.500,29194997,1,-1.0,...,2,한국,한국,한국예술종합학교 미디어콘텐츠센터,"(주)싸이더스,(주)블루라벨픽쳐스",12세이상관람가,드라마,신택수,"류현경,김주헌,김신비,김금순,공상아,이종윤,오재균,이수형,맹주원,송덕호,강주리,강정민",2023-01-07
26615,77,질펀한 마사지:대담한 손스킬,2023-01-07,6000,0.000,6000.0,1.000,6000,1,1.0,...,1,일본,일본,NaN,(주)영진크리에이티브,청소년관람불가,"성인물(에로),드라마,멜로/로맨스",디렉터 O,NaN,2023-01-07
26616,87,더 게임,2022-12-08,0,0.000,0.0,1.000,4450699,0,0.0,...,1,헝가리,헝가리,NaN,알토미디어(주),15세이상관람가,"드라마,스릴러",페테르 퍼자카스,가브리엘라 하모리,2023-01-07


In [25]:
df_final.to_csv("c:/2nd_project/Data/movie_data/2023_0107_merge.csv", index=False)